<a href="https://colab.research.google.com/github/sergiocabrales/python/blob/main/Fundamental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Analyzing Financial Statements for Insight into Company's Valuation for Possible Investment
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

symbol = 'AAPL.NZ'
url = 'https://finance.yahoo.com/quote/AAPL/financials?p=AAPL'
# url = 'https://finance.yahoo.com/quote/AAPL/balance-sheet?p=AAPL'


# Set up the request headers that we're going to use, to simulate
# a request by the Chrome browser. Simulating a request from a browser
# is generally good practice when building a scraper

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
}

# Fetch the page that we're going to parse, using the request headers
# defined above

page = requests.get(url, headers)

# Parse the page with LXML, so that we can start doing some XPATH queries
# to extract the data that we want

tree = html.fromstring(page.content)

# Smoke test that we fetched the page by fetching and displaying the H1 element

tree.xpath("//h1/text()")

['Apple Inc. (AAPL)']

In [ ]:
table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

# Ensure that some table rows are found; if none are found, then it's possible
# that Yahoo Finance has changed their page layout, or have detected
# that you're scraping the page.

assert len(table_rows) > 0

parsed_rows = []

for table_row in table_rows:
    parsed_row = []
    el = table_row.xpath("./div")
    
    none_count = 0
    
    for rs in el:
        try:
            (text,) = rs.xpath('.//span/text()[1]')
            parsed_row.append(text)
        except ValueError:
            parsed_row.append(np.NaN)
            none_count += 1

    if (none_count < 4):
        parsed_rows.append(parsed_row)

df = pd.DataFrame(parsed_rows)
df

,0,1,2,3,4,5
0,Breakdown,ttm,9/30/2020,9/30/2019,9/30/2018,9/30/2017
1,Total Revenue,"273,857,000","274,515,000","260,174,000","265,595,000","229,234,000"
2,Cost of Revenue,"169,277,000","169,559,000","161,782,000","163,756,000","141,048,000"
3,Gross Profit,"104,580,000","104,956,000","98,392,000","101,839,000","88,186,000"
4,Operating Expense,"37,442,000","38,668,000","34,462,000","30,941,000","26,842,000"
5,Operating Income,"67,138,000","66,288,000","63,930,000","70,898,000","61,344,000"
6,Net Non Operating Interest Income Expense,"1,052,000","890,000","1,385,000","2,446,000","2,878,000"
7,Other Income Expense,"127,000","-87,000","422,000","-441,000","-133,000"
8,Pretax Income,"68,317,000","67,091,000","65,737,000","72,903,000","64,089,000"
9,Tax Provision,"9,893,000","9,680,000","10,481,000","13,372,000","15,738,000"


In [ ]:
#import libraries
import pandas as pd
import numpy as np
import datetime
import pandas_datareader as web
import pandas_datareader as pdr
import matplotlib.pyplot as plt

In [ ]:
# Get the stock quote
df = web.DataReader('EC', data_source='yahoo',start='2020-11-01', end='2021-03-08')
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-11-02,9.44,9.15,9.38,9.36,869100,9.36
2020-11-03,9.68,9.39,9.51,9.54,927800,9.54
2020-11-04,9.89,9.44,9.55,9.73,805100,9.73
2020-11-05,10.06,9.73,9.73,9.80,672200,9.80
2020-11-06,9.97,9.76,9.76,9.80,1008800,9.80
...,...,...,...,...,...,...
2021-03-02,12.53,12.26,12.41,12.34,792300,12.34
2021-03-03,12.47,12.27,12.37,12.34,1245800,12.34
2021-03-04,12.87,12.32,12.42,12.52,1761800,12.52


In [ ]:
# Calculate P/BV (Price to Book Value)
